In [6]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
import kaggle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


import os
import composeml as cp
import featuretools as ft
import dask.dataframe as dd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

### Importing and preparing the data

In [7]:
#Convert csv to DataFrame
oct19 = pd.read_csv("data/2019-Oct.csv")
nov19 = pd.read_csv("data/2019-Nov.csv")
dec19 = pd.read_csv("data/2019-Dec.csv")
jan20 = pd.read_csv("data/2020-Jan.csv")
feb20 = pd.read_csv("data/2020-Feb.csv")

In [8]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [9]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


# Data Cleaning

### NAs and how to deal with them

In [10]:
#8757117 nas is Brand
#4598 nas in user_session
#20339246 nas in category_code DROPPED

In [11]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [12]:
#NAs in user_session 
ecom_df.dropna(inplace=True)
print(ecom_df.isna().sum())
print(ecom_df.shape)

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64
(353545, 9)


In [13]:
ecom_df.drop("category_code", axis=1, inplace = True)

In [14]:
ecom_df = ecom_df[ecom_df['price']>= 0]

### OHE (Multilabel Binarizer)

In [15]:
encoder = LabelBinarizer()
etype_labels = pd.DataFrame(encoder.fit_transform(ecom_df['event_type']), columns=encoder.classes_, index=ecom_df.index)
etype_labels.head()

,cart,purchase,remove_from_cart,view
21,0,0,0,1
264,0,0,1,0
269,0,0,1,0
274,0,0,1,0
276,0,0,1,0


In [16]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

,babyliss,benovy,concept,cutrin,depilflax,dewal,domix,emil,entity,estel,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
276,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Join our Labeled Columns to our main DataFrame
ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df.head()

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
21,2019-10-01 00:00:53 UTC,view,5856191,1487580006350586771,runail,24.44,507355498,944c7e9b-40bd-4112-a05b-81e73f37e0c0,0,0,...,0,0,0,1,0,0,0,0,0,0
33,2019-12-01 00:01:47 UTC,view,5732026,1487580011970953351,domix,4.16,509445281,91e9d610-759a-4831-b89a-22f2ab5fbc0f,0,0,...,0,0,0,0,0,0,0,0,0,0
52,2019-12-01 00:02:42 UTC,view,5746846,2193074740686488401,irisk,52.70,579970581,0b92a801-25d8-4da5-b514-fda26db3faae,0,0,...,0,0,0,0,0,0,0,0,0,0
66,2019-11-01 00:04:33 UTC,purchase,5766980,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0
67,2019-11-01 00:04:33 UTC,purchase,5767494,1487580013053083824,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0


# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

In [18]:
#df[['x','y','Category']].groupby(['Category']).count()

In [19]:
#ecom_df.purchase_this_session = ecom_df[['purchase']].count().grouby(['user_session'])

### Purchases_df

Created a dataframe of just purchases to do simplier features

In [20]:
purchases_df = ecom_df.loc[ecom_df.event_type=='purchase']
purchases_df.shape

(20538, 57)

In [40]:
#add column that adds total value of purchases for that user_session
purchases_df["session_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_session)["price"].cumsum()
#ecom_df['duplicate'] = ecom_df.groupby(cols)['user_id'].transform('size')
#can use cumsum to generate up to this point sums by user ID
purchases_df["lifetime_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_id)["price"].cumsum()
#purchases from session
purchases_df["purchases_in_session"] = purchases_df.groupby(purchases_df.user_id)["user_session"].transform('count')
#add event_type numericall
purchases_df["event_type"]=0
purchases_df.head()

,event_time,product_id,category_id,brand,price,user_id,user_session,cart,purchase,remove_from_cart,...,shifei,shik,unknown,vosev,yoko,ypsed,session_cumulative_purchases,lifetime_cumulative_purchases,purchases_in_session,event_type
66,2019-11-01 00:04:33 UTC,5766980,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,0,1.98,1.98,4,0
67,2019-11-01 00:04:33 UTC,5767494,1487580013053083824,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,0,4.12,4.12,4,0
68,2019-11-01 00:04:33 UTC,5775813,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,0,6.10,6.10,4,0
69,2019-11-01 00:04:33 UTC,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,0,8.08,8.08,4,0
641,2019-11-01 00:46:37 UTC,5749199,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,0,...,0,0,0,0,0,0,1.98,1.98,6,0


### Removed_df

Created a dataframe of just removed to do simplier features

In [39]:
removed_df = ecom_df.loc[ecom_df.event_type=="remove_from_cart"]
removed_df["event_type"]=1
removed_df.head()

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
264,2019-10-01 00:07:58 UTC,1,5767493,1487580013053083824,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,0,0,...,0,0,0,0,0,0,0,0,0,0
264,2019-10-01 00:07:58 UTC,1,5767493,1487580013053083824,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,0,0,...,0,0,0,0,0,0,1,0,0,0
264,2019-10-01 00:07:58 UTC,1,5767493,1487580013053083824,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,0,0,...,0,0,0,0,0,0,0,0,0,0
264,2019-10-01 00:07:58 UTC,1,5767493,1487580013053083824,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,0,0,...,0,0,0,0,0,0,1,0,0,0
269,2019-10-01 00:08:03 UTC,1,5759489,1487580013053083824,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,0,0,...,0,0,0,0,0,0,0,0,0,0


# LETS TRY AN FSM SHRUGS

10/17 10pm update theres an issue with the merge thats causing the classes to become a single class. Will fix this tomorrow zZz

In [35]:
temp_df = ecom_df[ecom_df.event_type != 'view']
temp_df = temp_df[temp_df.event_type != 'cart']
print(temp_df.event_type.value_counts())
print(temp_df.shape)

remove_from_cart    46263
purchase            20538
Name: event_type, dtype: int64
(66801, 57)


In [43]:
#purchases_df.drop("event_type", axis=1, inplace=True)
#removed_df.drop("event_type", axis=1, inplace=True)
dataframes=[purchases_df, removed_df]
fsm = pd.concat(dataframes)

In [48]:
#quick nans fix for missing columns from removed_df
fsm['session_cumulative_purchases'] = fsm['session_cumulative_purchases'].fillna(0)
fsm['purchases_in_session'] = fsm['purchases_in_session'].fillna(0)
fsm['lifetime_cumulative_purchases'] = fsm['lifetime_cumulative_purchases'].fillna(0)
fsm.sample(10)

,event_time,product_id,category_id,brand,price,user_id,user_session,cart,purchase,remove_from_cart,...,shifei,shik,unknown,vosev,yoko,ypsed,session_cumulative_purchases,lifetime_cumulative_purchases,purchases_in_session,event_type
896448,2019-11-07 14:48:14 UTC,5712695,2007399943458784057,nitrimax,10.32,491024472,16f240bf-4a81-5532-f3c9-fd3b18fef24d,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
2613723,2020-02-18 17:24:21 UTC,5788781,1487580013053083824,italwax,1.98,319252541,3b4c28c4-dd2f-405b-97d6-2a65f3edffa0,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
2414093,2020-01-19 15:14:12 UTC,8098,1487580006350586771,unknown,1.11,579890584,a9733712-7e05-4d58-b745-13e762a5c4a3,0,1,0,...,0,0,1,0,0,0,25.71,25.71,2.0,0
2554929,2019-10-19 18:12:13 UTC,6817,2007399943458784057,unknown,5.24,561203232,379e07a1-7e0c-4104-b952-fc615919776b,0,0,1,...,0,0,1,0,0,0,0.00,0.00,0.0,1
24421,2020-01-01 12:21:13 UTC,24331,1487580013053083824,depilflax,3.02,531210052,f72d5aa6-1429-4865-bc9a-595e69da8d6c,0,0,0,...,0,0,0,0,0,0,3.02,3.02,6.0,0
1657398,2020-01-14 09:14:01 UTC,5873430,2007399943458784057,benovy,7.46,600957813,c8e71fa6-669b-4e46-89c0-d30ef973f20b,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
3621600,2020-01-27 19:07:16 UTC,24331,1487580013053083824,depilflax,3.02,542693330,6c833c29-e7d1-4241-9a45-ccbf1e5c55e9,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
2844620,2019-12-23 15:28:45 UTC,5712700,2007399943458784057,nitrimax,10.16,516716310,81274e24-2028-4e72-9115-c01cf70920ab,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
82310,2020-02-01 15:13:12 UTC,5766980,1487580013053083824,italwax,1.98,602921137,d7775595-3e9f-4611-95b3-8256143ad328,0,0,1,...,0,0,0,0,0,0,0.00,0.00,0.0,1
82081,2020-01-02 00:39:41 UTC,5756536,2007399943458784057,unknown,6.19,562690676,245bcb98-5e2b-41ea-bd7a-5d14c0682b09,0,0,1,...,0,0,1,0,0,0,0.00,0.00,0.0,1


In [54]:
#HERE WE ARE LOOSING ALL MY CLASSES
le = preprocessing.LabelEncoder()

le.fit(ecom_df['event_type'])

ecom_df['event_type']=le.transform(ecom_df['event_type'])

ecom_df.event_type.value_counts()

3    275072
0     81704
2     46263
1     20538
Name: event_type, dtype: int64

# FSM

This is a proof of concept not I will spend my time feature engineering to make this model better. I have started some of it above.

In [55]:
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = temp_df[temp_df.event_type != 'cart']
ecom_df.sample()


,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
1267166,2019-10-09 11:37:07 UTC,remove_from_cart,5732026,1487580011970953351,domix,4.16,544390138,84b8c851-42ee-4802-8ab2-7103b56d3ff0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [56]:
#"total_visits", "total_purchases_session"
X = ecom_df.drop(["purchase", "cart", "purchase", "remove_from_cart", "view", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [57]:
X.sample(20)
#why is their cumulative purchases in session in where nothing was bought

,product_id,category_id,price,user_id,babyliss,benovy,concept,cutrin,depilflax,dewal,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
1817243,5766980,1487580013053083824,1.98,512487929,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1642951,5861764,1487580006350586771,1.43,608105377,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3838608,5803579,1487580013053083824,2.14,573377821,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3137300,5884578,1487580006350586771,63.33,529500607,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
904024,5712695,2007399943458784057,10.16,459557183,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
308679,5901859,2007399943458784057,5.62,438912888,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2383466,5901858,2007399943458784057,5.62,586420614,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4012689,8098,1487580006350586771,1.11,472703743,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2040507,5861760,1487580006350586771,71.43,367016316,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2541182,5749198,1487580013053083824,1.98,603703710,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
import lightgbm as lgb

In [60]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train, y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
print(classification_report(y_test, test_preds_lgbm))

                  precision    recall  f1-score   support

        purchase       0.68      0.05      0.09      4123
remove_from_cart       0.70      0.99      0.82      9238

        accuracy                           0.70     13361
       macro avg       0.69      0.52      0.46     13361
    weighted avg       0.69      0.70      0.60     13361



Looking at this dataset, if the customer ids are not unique and you are able to track customer behavior over time there is an opportunity to use feature engineering to show how many times a customer has done something to predict whether or not the customer will purchase the product.  Ie, how many things were added to a cart within a given time frame, how many purchases has the customer completed in the past. That would make for a fine capstone.

# Purchases_df

In [120]:
purchases_df = ecom_df.loc[ecom_df.event_type=='purchase']

In [148]:
#add column that adds total value of purchases for that user_session
purchases_df["session_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_session)["price"].cumsum()
#ecom_df['duplicate'] = ecom_df.groupby(cols)['user_id'].transform('size')
#can use cumsum to generate up to this point sums by user ID
purchases_df["lifetime_cumulative_purchases"] = purchases_df.groupby(purchases_df.user_id)["price"].cumsum()
#total purchases from session
#purchases_df["total_purchases_session"] = purchases_df.groupby("user_session")["price"].transform('sum')

In [136]:
#total vitis
purchases_df["purchases_in_session"] = purchases_df.groupby(purchases_df.user_id)["user_session"].transform('count')
purchases_df["total_visits"] = purchases_df.groupby(purchases_df.user_id)["user_session"].unique().count()
purchases_df.head(10)


,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,shik,unknown,vosev,yoko,ypsed,session_cumulative_purchases,lifetime_cumulative_purchases,total_purchases_session,total_visits,purchases_in_session
66,2019-11-01 00:04:33 UTC,purchase,5766980,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,1.98,1.98,8.08,NaN,4
67,2019-11-01 00:04:33 UTC,purchase,5767494,1487580013053083824,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,4.12,4.12,8.08,NaN,4
68,2019-11-01 00:04:33 UTC,purchase,5775813,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,6.10,6.10,8.08,NaN,4
69,2019-11-01 00:04:33 UTC,purchase,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,8.08,8.08,8.08,NaN,4
641,2019-11-01 00:46:37 UTC,purchase,5749199,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,...,0,0,0,0,0,1.98,1.98,5.94,NaN,6
644,2019-11-01 00:46:37 UTC,purchase,5775814,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,...,0,0,0,0,0,3.96,3.96,5.94,NaN,6
645,2019-11-01 00:46:37 UTC,purchase,5788783,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,...,0,0,0,0,0,5.94,5.94,5.94,NaN,6
1059,2019-10-01 00:40:00 UTC,purchase,5884577,1487580006350586771,max,47.46,509896854,e62d29f3-b14a-4c25-afb0-ddbdea4bcf89,0,1,...,0,0,0,0,0,47.46,47.46,56.98,NaN,2
1060,2019-10-01 00:40:00 UTC,purchase,5889696,2007399943458784057,nitrile,9.52,509896854,e62d29f3-b14a-4c25-afb0-ddbdea4bcf89,0,1,...,0,0,0,0,0,56.98,56.98,56.98,NaN,2
1353,2019-10-01 00:55:17 UTC,purchase,6737,2007399943458784057,unknown,5.24,446404021,abac4e23-d977-47d9-99e9-7faf35e62cc1,0,1,...,0,1,0,0,0,5.24,5.24,5.24,NaN,1


timer series, agg data, dont rpedict indivdual session, forecasting purchases.

class, if they purchased or removed, cart, viewed. create a model that when something is added to the cart, wther they purchased or removed.

In [216]:
purchases_df["total_visits"] = purchases_df.groupby(purchases_df.user_id)["user_session"].transform('count')
purchases_df.head(10)
#create unique list of customer ids
#loop over and create a dataframe of observations for that user_id
#.unique on the user_sessions
#len() it
#create dict with user_id as key, and number of unique sessions as value.
#map that. and feed in that dict, which will output an array for each row.

,event_time,product_id,category_id,brand,price,user_id,user_session,cart,purchase,remove_from_cart,...,shik,unknown,vosev,yoko,ypsed,session_cumulative_purchases,lifetime_cumulative_purchases,total_purchases_session,total_visits,purchases_in_session
66,2019-11-01 00:04:33 UTC,5766980,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,1.98,1.98,8.08,4,4
67,2019-11-01 00:04:33 UTC,5767494,1487580013053083824,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,4.12,4.12,8.08,4,4
68,2019-11-01 00:04:33 UTC,5775813,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,6.10,6.10,8.08,4,4
69,2019-11-01 00:04:33 UTC,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,0,...,0,0,0,0,0,8.08,8.08,8.08,4,4
641,2019-11-01 00:46:37 UTC,5749199,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,0,...,0,0,0,0,0,1.98,1.98,5.94,6,6
644,2019-11-01 00:46:37 UTC,5775814,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,0,...,0,0,0,0,0,3.96,3.96,5.94,6,6
645,2019-11-01 00:46:37 UTC,5788783,1487580013053083824,italwax,1.98,457604149,34adbc64-eebc-4d45-8542-547b9c20f6c1,0,1,0,...,0,0,0,0,0,5.94,5.94,5.94,6,6
1059,2019-10-01 00:40:00 UTC,5884577,1487580006350586771,max,47.46,509896854,e62d29f3-b14a-4c25-afb0-ddbdea4bcf89,0,1,0,...,0,0,0,0,0,47.46,47.46,56.98,2,2
1060,2019-10-01 00:40:00 UTC,5889696,2007399943458784057,nitrile,9.52,509896854,e62d29f3-b14a-4c25-afb0-ddbdea4bcf89,0,1,0,...,0,0,0,0,0,56.98,56.98,56.98,2,2
1353,2019-10-01 00:55:17 UTC,6737,2007399943458784057,unknown,5.24,446404021,abac4e23-d977-47d9-99e9-7faf35e62cc1,0,1,0,...,0,1,0,0,0,5.24,5.24,5.24,1,1


In [147]:
ecom_df[ecom_df.user_session == list(ecom_df.user_session)[3]]

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,cart,purchase,...,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko,ypsed
66,2019-11-01 00:04:33 UTC,purchase,5766980,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0
67,2019-11-01 00:04:33 UTC,purchase,5767494,1487580013053083824,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0
68,2019-11-01 00:04:33 UTC,purchase,5775813,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0
69,2019-11-01 00:04:33 UTC,purchase,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,1,...,0,0,0,0,0,0,0,0,0,0
4097968,2019-10-31 21:58:22 UTC,cart,5775813,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,1,0,...,0,0,0,0,0,0,0,0,0,0
4101291,2019-10-31 23:14:24 UTC,view,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,0,0,...,0,0,0,0,0,0,0,0,0,0
4101556,2019-10-31 23:24:49 UTC,cart,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,1,0,...,0,0,0,0,0,0,0,0,0,0
4101556,2019-10-31 23:24:49 UTC,cart,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,1,0,...,0,0,0,0,0,0,1,0,0,0
4101556,2019-10-31 23:24:49 UTC,cart,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,1,0,...,0,0,0,0,0,0,0,0,0,0
4101556,2019-10-31 23:24:49 UTC,cart,5775814,1487580013053083824,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,1,0,...,0,0,0,0,0,0,1,0,0,0


In [155]:
ecom_df.event_type.value_counts()

view                275072
cart                 81704
remove_from_cart     46263
purchase             20538
Name: event_type, dtype: int64

In [112]:
len(test[test.index <= test.index[11]])

16

In [94]:
#add column that adds total number of purchases in the dataset
purchases_df = ecom_df.query('event_type=="purchase"').groupby(['user_id']).size().reset_index(name='count')
purchases_df.head()
#merge purchases_df with main
ecom_df.merge(purchases_df, on="user_id")

,user_id,count
0,9794320,4
1,10079204,2
2,10280338,86
3,12055855,4
4,12936739,2


# Removed_df

In [211]:
removed_df = ecom_df.loc[ecom_df.event_type=="remove_from_cart"]
removed_df.shape

(46263, 57)

In [ ]:
#create a dataframe with customer id and first purchase date in tx_next
ecom_df_first_purchase = ecom_df.groupby('user_id').event_time.min().reset_index()
ecom_df_first_purchase.columns = ['user_id','MinPurchaseDate']

#create a dataframe with customer id and last purchase date in tx_6m
ecom_df_last_purchase = tx_6m.groupby('user_id').InvoiceDate.max().reset_index()
ecom_df_last_purchase.columns = ['user_id','MaxPurchaseDate']

Predicting future sales of a product_ID based on sales

In [121]:
len(ecom_df.product_id.unique())

54566

product ids as index

price

number of sales per day, forecasting sales

if a customer puts an item in their cart 

how likely are they to purchase

when was it added to the cart total

out of all the times was it added to the cart ho wmany times was it purchased.

product_ids is index

of all the times a product in the cart how many times was it purchased.